Pandas allows us to treat data in a table-like way (think excel) and to utilize a structure called a Data Frame. These methods make it much easier to manipulate and slice data. No more excessive loops and list comprehensions. 

In [2]:
import pandas as pd

In [5]:
users = pd.read_table('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user',  sep='|', index_col='user_id')

In [ ]:
users

In [8]:
type(users) # Date Frame - pandas is all about this type of object

pandas.core.frame.DataFrame

In [9]:
users.head(5) 

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [10]:
users.columns

Index([u'age', u'gender', u'occupation', u'zip_code'], dtype='object')

In [13]:
users.index # The "index" or the row labels

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            934, 935, 936, 937, 938, 939, 940, 941, 942, 943],
           dtype='int64', name=u'user_id', length=943)

In [15]:
users.shape

(943, 4)

In [18]:
users.values # so there's an underlying "array" data type, and the Data Frame object encapsulates it, and adds more functionality

array([[24, 'M', 'technician', '85711'],
       [53, 'F', 'other', '94043'],
       [23, 'M', 'writer', '32067'],
       ..., 
       [20, 'M', 'student', '97229'],
       [48, 'F', 'librarian', '78209'],
       [22, 'M', 'student', '77841']], dtype=object)

With a Data Frame, we have columns with names, so we can select on those columns.

In [ ]:
# select a column
users['gender']         # select one column
users.gender            # or with dot notation

In [22]:
type(users.gender) # a Series is a basic data type in pandas

pandas.core.series.Series

Data Frames have many built-in functions for things we might commonly want. For example, if we want a statistical summary of the numeric columns.

In [23]:
# summarize (describe) the DataFrame
users.describe()                    # describe all numeric columns


,age
count,943.000000
mean,34.051962
std,12.192740
min,7.000000
25%,25.000000
50%,31.000000
75%,43.000000
max,73.000000


In [24]:
# or a statistical summary of all columns
users.describe(include='all') 

,age,gender,occupation,zip_code
count,943.000000,943,943,943
unique,NaN,2,21,795
top,NaN,M,student,55414
freq,NaN,670,196,9
mean,34.051962,NaN,NaN,NaN
std,12.192740,NaN,NaN,NaN
min,7.000000,NaN,NaN,NaN
25%,25.000000,NaN,NaN,NaN
50%,31.000000,NaN,NaN,NaN
75%,43.000000,NaN,NaN,NaN


For any column, we can access useful statistical summaries.

In [ ]:
users.age.mean()

In [25]:
users.gender.value_counts()

M    670
F    273
Name: gender, dtype: int64

** Exercise** 

1. Read drinks.csv into a DataFrame called 'drinks'
2. Print the head and the tail 
3. Examine the default index, data types, and shape
4. Print the 'beer_servings' Series
5. Calculate the mean 'beer_servings' for the entire dataset
6. Count the number of occurrences of each 'continent' value
7. **Bonus** Display only the number of rows of the 'users' DataFrame
8. **Bonus** Display the 3 most frequent occupations in 'users'

Pandas provides easy functionality for a variety of filtering and sorting tasks we inevitably encounter. Again, this kind of work could be done with loops and conditionals, but the Data Frame methods make things much easier.

For example, we can filter on a boolean expression:

In [26]:
#  boolean filtering: only show users with age < 20
young_bool = users.age < 20         # create a Series of booleans...
users[young_bool]                   # ...and use that Series to filter rows
users[users.age < 20]               # or, combine into a single step
users[users.age < 20].occupation    # select one column from the filtered results
users[users.age < 20].occupation.value_counts()     # value_counts of resulting Series

student          64
other             4
none              3
writer            2
entertainment     2
salesman          1
artist            1
Name: occupation, dtype: int64

And we can have filtering with multiple conditions:

In [ ]:
# boolean filtering with multiple conditions
users[(users.age < 20) & (users.gender=='M')]       # ampersand for AND condition
users[(users.age < 20) | (users.age > 60)]          # pipe for OR condition

And we can sort by various columns:

In [ ]:
# sorting
users.age.order()                   # sort a column
users.sort('age')                   # sort a DataFrame by a single column
users.sort('age', ascending=False)  # use descending order instead

** Exercise** 

1. Filter 'drinks' to only include European countries
2. Filter 'drinks' to only include European countries with wine_servings > 300
3. Calculate the mean 'beer_servings' for all of Europe
4. Determine which 10 countries have the highest total_litres_of_pure_alcohol
5. **Bonus** Sort 'users' by 'occupation' and then by 'age' (in a single command)

We can easily rename columns if needed. And it is easy to remove existing columns or to add new ones.

In [29]:
drinks = pd.read_csv("../data/drinks.csv")
# rename one or more columns
drinks.rename(columns={'beer_servings':'beer', 'wine_servings':'wine'})
drinks.rename(columns={'beer_servings':'beer', 'wine_servings':'wine'}, inplace=True)

In [ ]:
# replace all column names
drink_cols = ['country', 'beer', 'spirit', 'wine', 'liters', 'continent']
drinks.columns = drink_cols

In [ ]:
# replace all column names when reading the file
drinks = pd.read_csv('drinks.csv', header=0, names=drink_cols)


In [ ]:
# add a new column as a function of existing columns
drinks['servings'] = drinks.beer + drinks.spirit + drinks.wine
drinks['mL'] = drinks.liters * 1000

In [ ]:
# removing columns
drinks.drop('mL', axis=1)                               # axis=0 for rows, 1 for columns
drinks.drop(['mL', 'servings'], axis=1, inplace=True)   # drop multiple columns

In data analysis, we often encounter missing data and need to think about how to handle this. Many methods will simply ignore missing data, and we just need to make sure we're handling things sensibly. 

In [30]:
# missing values are usually excluded by default
drinks.continent.value_counts()              # excludes missing values
drinks.continent.value_counts(dropna=False)  # includes missing values

AF     53
EU     45
AS     44
NaN    23
OC     16
SA     12
Name: continent, dtype: int64

In [ ]:
# find missing values in a Series
drinks.continent.isnull()           # True if missing
drinks.continent.notnull()          # True if not missing

In [ ]:
# use a boolean Series to filter DataFrame rows
drinks[drinks.continent.isnull()]   # only show rows where continent is missing
drinks[drinks.continent.notnull()]  # only show rows where continent is not missing

In [ ]:
# side note: understanding axes
drinks.sum()            # sums "down" the 0 axis (rows)
drinks.sum(axis=0)      # equivalent (since axis=0 is the default)
drinks.sum(axis=1)      # sums "across" the 1 axis (columns)

In [ ]:
# side note: adding booleans
pd.Series([True, False, True])          # create a boolean Series
pd.Series([True, False, True]).sum()    # converts False to 0 and True to 1

In [ ]:
# find missing values in a DataFrame
drinks.isnull()             # DataFrame of booleans
drinks.isnull().sum()       # count the missing values in each column

In [ ]:
# drop missing values
drinks.dropna()             # drop a row if ANY values are missing
drinks.dropna(how='all')    # drop a row only if ALL values are missing

In [ ]:
# fill in missing values
drinks.continent.fillna(value='NA', inplace=True)   # fill in missing values with 'NA'

In [ ]:
# turn off the missing value filter
drinks = pd.read_csv('drinks.csv', header=0, names=drink_cols, na_filter=False)

** Exercise** 

1. Read ufo.csv into a DataFrame called 'ufo'
2. Check the shape of the DataFrame
3. Calculate the most frequent value for each of the columns (in a single command)
4. What are the four most frequent colors reported?
5. For reports in VA, what's the most frequent city?
6. Show only the UFO reports from Arlington, VA
7. Count the number of missing values in each column
8. Show only the UFO reports in which the City is missing
9. How many rows remain if you drop all rows with any missing values?
10. **Bonus** Create a new column called 'Location' that includes both City and State
11. **Bonus** Replace any spaces in the column names with an underscore

We can perform operations on Data Frame subsets and then record only the result for each subset. Think about this as a [split-apply-combine](http://i.imgur.com/yjNkiwL.png) operation. Or perhaps you're familiar with the notion of "Group By".

In [ ]:
# for each continent, calculate the mean beer servings
drinks.groupby('continent').beer.mean()

In [ ]:
# for each continent, count the number of occurrences
drinks.continent.value_counts()

In [ ]:
# for each continent, describe beer servings
drinks.groupby('continent').beer.describe()

In [ ]:
# similar, but outputs a DataFrame and can be customized
drinks.groupby('continent').beer.agg(['count', 'mean', 'min', 'max'])
drinks.groupby('continent').beer.agg(['count', 'mean', 'min', 'max']).sort('mean')

In [ ]:
# if you don't specify a column to which the aggregation function should be applied,
# it will be applied to all numeric columns
drinks.groupby('continent').mean()
drinks.groupby('continent').describe()

** Exercise **

1. For each occupation in 'users', count the number of occurrences
2. For each occupation, calculate the mean age
3. **Bonus** For each occupation, calculate the minimum and maximum ages
4. **Bonus** For each combination of occupation and gender, calculate the mean age
5. Think about how obnoxious it would be to do all this stuff with a for loop. 

Also, note that we can select multiple columns or rows when we're going this kind of subsetting. 

In [ ]:
# select multiple columns
my_cols = ['City', 'State']     # create a list of column names...
ufo[my_cols]                    # ...and use that list to select columns
ufo[['City', 'State']]          # or, combine into a single step

In [ ]:
# use loc to select columns by name
ufo.loc[:, 'City']              # colon means "all rows", then select one column
ufo.loc[:, ['City', 'State']]   # select two columns
ufo.loc[:, 'City':'State']      # select a range of columns

In [ ]:
# loc can also filter rows by "name" (the index)
ufo.loc[0, :]                   # row 0, all columns
ufo.loc[0:2, :]                 # rows 0/1/2, all columns
ufo.loc[0:2, 'City':'State']    # rows 0/1/2, range of columns

In [ ]:
# use iloc to filter rows and select columns by integer position
ufo.iloc[:, [0, 3]]             # all rows, columns in position 0/3
ufo.iloc[:, 0:4]                # all rows, columns in position 0/1/2/3
ufo.iloc[0:3, :]                # rows in position 0/1/2, all columns

Here's a grab-bag of other common Data Frame features that you'll commonly use. 

In [ ]:
# map existing values to a different set of values
users['is_male'] = users.gender.map({'F':0, 'M':1})

In [ ]:
# encode strings as integer values (automatically starts at 0)
users['occupation_num'] = users.occupation.factorize()[0]

In [ ]:
# determine unique values in a column
users.occupation.nunique()      # count the number of unique values
users.occupation.unique()       # return the unique values

In [ ]:
# replace all instances of a value in a column (must match entire value)
ufo.State.replace('Fl', 'FL', inplace=True)

In [ ]:
# string methods are accessed via 'str'
ufo.State.str.upper()                               # converts to uppercase
ufo.Colors_Reported.str.contains('RED', na='False') # checks for a substring

In [ ]:
# convert a string to the datetime format
ufo['Time'] = pd.to_datetime(ufo.Time)
ufo.Time.dt.hour                        # datetime format exposes convenient attributes
(ufo.Time.max() - ufo.Time.min()).days  # also allows you to do datetime "math"
ufo[ufo.Time > pd.datetime(2014, 1, 1)] # boolean filtering with datetime format


In [ ]:
# setting and then removing an index
ufo.set_index('Time', inplace=True)
ufo.reset_index(inplace=True)

In [ ]:
# sort a column by its index
ufo.State.value_counts().sort_index()

In [ ]:
# change the data type of a column
drinks['beer'] = drinks.beer.astype('float')

In [ ]:
# change the data type of a column when reading in a file
pd.read_csv('drinks.csv', dtype={'beer_servings':float})

In [ ]:
# create dummy variables for 'continent' and exclude first dummy column
continent_dummies = pd.get_dummies(drinks.continent, prefix='cont').iloc[:, 1:]

In [ ]:
# concatenate two DataFrames (axis=0 for rows, axis=1 for columns)
drinks = pd.concat([drinks, continent_dummies], axis=1)

Here's some less-frequently-used stuff that's good to know about.

In [ ]:
# create a DataFrame from a dictionary
pd.DataFrame({'capital':['Montgomery', 'Juneau', 'Phoenix'], 'state':['AL', 'AK', 'AZ']})

In [ ]:
# create a DataFrame from a list of lists
pd.DataFrame([['Montgomery', 'AL'], ['Juneau', 'AK'], ['Phoenix', 'AZ']], columns=['capital', 'state'])

In [ ]:
# detecting duplicate rows
users.duplicated()          # True if a row is identical to a previous row
users.duplicated().sum()    # count of duplicates
users[users.duplicated()]   # only show duplicates
users.drop_duplicates()     # drop duplicate rows
users.age.duplicated()      # check a single column for duplicates
users.duplicated(['age', 'gender', 'zip_code']).sum()   # specify columns for finding duplicates

In [ ]:
# display a cross-tabulation of two Series
pd.crosstab(users.occupation, users.gender)

In [ ]:
# alternative syntax for boolean filtering (noted as "experimental" in the documentation)
users.query('age < 20')                 # users[users.age < 20]
users.query("age < 20 and gender=='M'") # users[(users.age < 20) & (users.gender=='M')]
users.query('age < 20 or age > 60')     # users[(users.age < 20) | (users.age > 60)]

In [ ]:
# display the memory usage of a DataFrame
ufo.info()          # total usage
ufo.memory_usage()  # usage by column

In [ ]:
# change a Series to the 'category' data type (reduces memory usage and increases performance)
ufo['State'] = ufo.State.astype('category')

In [ ]:
# temporarily define a new column as a function of existing columns
drinks.assign(servings = drinks.beer + drinks.spirit + drinks.wine)


In [ ]:
# limit which rows are read when reading in a file
pd.read_csv('drinks.csv', nrows=10)           # only read first 10 rows
pd.read_csv('drinks.csv', skiprows=[1, 2])    # skip the first two rows of data

In [ ]:
# write a DataFrame out to a CSV
drinks.to_csv('drinks_updated.csv')                 # index is used as first column
drinks.to_csv('drinks_updated.csv', index=False)    # ignore index

In [ ]:
# save a DataFrame to disk (aka 'pickle') and read it from disk (aka 'unpickle')
drinks.to_pickle('drinks_pickle')
pd.read_pickle('drinks_pickle')

In [ ]:
# randomly sample a DataFrame
train = drinks.sample(frac=0.75, random_state=1)    # will contain 75% of the rows
test = drinks[~drinks.index.isin(train.index)]      # will contain the other 25%

In [ ]:
# change the maximum number of rows and columns printed ('None' means unlimited)
pd.set_option('max_rows', None)     # default is 60 rows
pd.set_option('max_columns', None)  # default is 20 columns
print drinks

In [ ]:
# reset options to defaults
pd.reset_option('max_rows')
pd.reset_option('max_columns')

That's a lot of litte odds-and-ends, but I hope you realize that the Data Frame idea is superbly useful will save us a lot of work.